In [7]:
import numpy as np
import pandas as pd
import seaborn as sn
import os
import sklearn as sk
import statsmodels.api as sm

class country:
    def __init__(self,name,code_country,year,z_used='NA'):
        self.country_name = name
        self.year = year
        self.code_country = code_country
        url_base = 'https://github.com/mermozhk/Poverty/raw/main/'
        #print('/'.join([url_base,name,year,'ehcvm_welfare_'+''.join([code_country,year])])+'.dta')
        self.welfare_data = pd.read_stata('/'.join([url_base,name,year,'ehcvm_welfare_'+''.join([code_country,year])])+'.dta')
        self.welfare_data['dtot_corrected']=self.welfare_data['dtot']/self.welfare_data['def_spa']
        
        self.conso_data = pd.read_stata('/'.join([url_base,name,year,'ehcvm_conso_'+''.join([code_country,year])])+'.dta')
        self.menage_data = pd.read_stata('/'.join([url_base,name,year,'ehcvm_menage_'+''.join([code_country,year])])+'.dta')
        self.individu_data = pd.read_stata('/'.join([url_base,name,year,'ehcvm_individu_'+''.join([code_country,year])])+'.dta')
        if z_used=='NA':
            self.merge_data = pd.merge(self.individu_data,
                   self.welfare_data[['hhid','zref','dtot_corrected','hhsize']],
                   on = 'hhid',
                   how='outer')
        else:
            self.merge_data = pd.merge(self.individu_data,
                   self.welfare_data[['hhid','dtot_corrected','hhsize']],
                   on = 'hhid',
                   how='outer')
            self.merge_data['zref']=float(z_used)
            
        self.merge_data['constant'] =1
    
    
    def estimation_procedure(self,index):
        wls_model = sm.WLS(self.merge_data[index],self.merge_data['constant'], weights=self.merge_data['hhweight'])
        return wls_model.fit()
        
    def prevalence(self):
        self.merge_data['prevalence']=(self.merge_data['dtot_corrected']<
                                       (self.merge_data['zref']*self.merge_data['hhsize'])).map({False:0, True:1})
        return self.estimation_procedure('prevalence')
    
    def gap(self):
        self.merge_data['gap'] = (1-self.merge_data['dtot_corrected']/(self.merge_data['zref']*self.merge_data['hhsize']))*self.merge_data['prevalence']
        return self.estimation_procedure('gap')
    
    def severity(self):
        if not('prevalence' in self.merge_data.columns):
            
            self.merge_data['prevalence']=(self.merge_data['dtot_corrected']<
                                       (self.merge_data['zref']*self.merge_data['hhsize'])).map({False:0, True:1})
            
        self.merge_data['severity'] = ((1-self.merge_data['dtot_corrected']/(self.merge_data['zref']*self.merge_data['hhsize']))**2)*self.merge_data['prevalence']
        
        return self.estimation_procedure('severity')
    
    def aart_welfare_index(self):
        self.merge_data['aart_welfare'] = (self.merge_data['zref']*self.merge_data['hhsize'])/self.merge_data['dtot_corrected']
        return self.estimation_procedure('aart_welfare')
    
    def aart_poverty_index(self):
        if not('aart_welfare' in self.merge_data.columns):
            self.merge_data['aart_welfare'] = (self.merge_data['zref']*self.merge_data['hhsize'])/self.merge_data['dtot_corrected']
        
        self.merge_data['aart_poverty'] = self.merge_data.apply(lambda x: max(x.aart_welfare,1),axis=1)
        return self.estimation_procedure('aart_poverty')
        

In [13]:
Benin = country(name='Benin',code_country='BEN',year='2018')

In [11]:
Benin.aart_welfare_index().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:           aart_welfare   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 30 Mar 2024   Prob (F-statistic):                nan
Time:                        11:51:57   Log-Likelihood:                -16592.
No. Observations:               42343   AIC:                         3.319e+04
Df Residuals:                   42342   BIC:                         3.319e+04
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       0.5741      0.002    347.029      0.000       0.571       0.577
==============================================================================
Omnibus:                    21160.849   Durbin-Watson:                   0.224
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           377293.460
Skew:                           1.984   Prob(JB):                         0.00
Kurtosis:                      17.075   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
Benin.aart_poverty_index().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:           aart_poverty   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 30 Mar 2024   Prob (F-statistic):                nan
Time:                        11:52:06   Log-Likelihood:                 20197.
No. Observations:               42343   AIC:                        -4.039e+04
Df Residuals:                   42342   BIC:                        -4.038e+04
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       1.0318      0.001   1487.071      0.000       1.030       1.033
==============================================================================
Omnibus:                    70206.012   Durbin-Watson:                   0.247
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        134381090.013
Skew:                          10.842   Prob(JB):                         0.00
Kurtosis:                     278.131   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
Benin.prevalence().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:             prevalence   R-squared:                      -0.000
Model:                            WLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 30 Mar 2024   Prob (F-statistic):                nan
Time:                        11:53:54   Log-Likelihood:                -31719.
No. Observations:               42343   AIC:                         6.344e+04
Df Residuals:                   42342   BIC:                         6.345e+04
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       0.3848      0.002    162.753      0.000       0.380       0.389
==============================================================================
Omnibus:                   257576.355   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5750.461
Skew:                           0.503   Prob(JB):                         0.00
Kurtosis:                       1.501   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
Benin.severity().summary()

0        246541.546875
1        246541.546875
2        246541.546875
3        246541.546875
4        246541.546875
             ...      
42338    246541.546875
42339    246541.546875
42340    246541.546875
42341    246541.546875
42342    246541.546875
Name: zref, Length: 42343, dtype: float32